In [1]:
import pandas as pd
import numpy as np

# LOAD DATA

In [2]:
df = pd.read_csv("../datasets/Titanic/train.csv")

In [3]:
df.head(n=5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# DATA PREPROCESSING

we do not require all the columns to predict the survivour. So we will drop some columns

In [4]:
drop_columns = ['Name','SibSp','Parch','Ticket','Cabin','Embarked']
df = df.drop(columns=drop_columns)
df.head(n=5)

,PassengerId,Survived,Pclass,Sex,Age,Fare
0,1,0,3,male,22.0,7.2500
1,2,1,1,female,38.0,71.2833
2,3,1,3,female,26.0,7.9250
3,4,1,1,female,35.0,53.1000
4,5,0,3,male,35.0,8.0500


Dataset has some NAN values lets correct them too

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   Fare         891 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 41.9+ KB


In [6]:
#Age columns has some null values so we will fill them with mean of all ages

In [7]:
df = df.fillna(value=np.mean(df['Age']),axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   Fare         891 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 41.9+ KB


Now we want sex column to be numerical so change it also

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
LE = LabelEncoder()
df["Sex"] = LE.fit_transform(df["Sex"])

In [11]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare
0,1,0,3,1,22.0,7.2500
1,2,1,1,0,38.0,71.2833
2,3,1,3,0,26.0,7.9250
3,4,1,1,0,35.0,53.1000
4,5,0,3,1,35.0,8.0500


We get our clean DF , now from this clean df extract input and output features

In [12]:
x_cols = ['PassengerId','Pclass','Sex','Age','Fare']
y_col = ['Survived']

X = df[x_cols]
Y = df[y_col]

In [13]:
print(type(X),type(Y))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


# ENTROPY AND INFORMATION GAIN

In [23]:
def entropy(col):
    #frequency 
    counts = np.unique(col,return_counts=True)
    #total count
    N = float(col.shape[0])
    #entropy
    ent  = 0.0
    for ix in counts[1]:
        pi = ix/N
        #formula for entropy
        ent+=-1*pi*np.log2(pi)
    return ent

In [41]:
def divide_data(x_data,fkey,fvalue):
    #work with pandas Data frame
    
    #here data is empty list
    x_right = pd.DataFrame([],columns=x_data.columns)
    x_left = pd.DataFrame([],columns=x_data.columns)
    
    for ix in range(x_data.shape[0]):
        val = x_data[fkey].loc[ix]
        if val>fvalue:
            x_right = x_right.append(x_data.iloc[ix])
        else:
            x_left = x_left.append(x_data.iloc[ix])
    return x_left,x_right

In [44]:
# function to check divide_data function
# x_left,x_right = divide_data(X[:10],'Sex',0.5)
# print(x_left)
# print(x_right)

In [ ]:
#feature that gives us maximum information gain will be our Root node

# -fval is threshold value over which we want to divide decision eg if income 
# is greater than 5 lacs then 5 lac is threshold 5

def information_gain(x_data,fkey,fvalue):
    #divide data into left and right nodes
    left,right = divide_data(x_data,fkey,fvalue)
    
    # %of total sample are on left and right 
    l = float(left.shape[0]/x_data.shape[0])
    r = float(right.shape[0]/x_data.shape[0])
    
    #if all samples are on on side i.e either left or right
    if left.shape[0]==0 or right.shape[0]==0
        return -100000000 #minimize the IG so that it wont get selected
    
    i_gain = entrop